In [1]:
# add de folder to path
import sys
sys.path.insert(1, '../models/')
sys.path.insert(1, '../de/')
sys.path.insert(1, '../controller/')

import numpy as np
from scipy.linalg import expm
from qiskit.quantum_info.operators import Operator
from models import BaseModel
from signals import Constant, ConstantSignal, PiecewiseConstant
from DE_Problems import BMDE_Problem, SchrodingerProblem
from DE_Solvers import BMDE_Solver
from DE_Options import DE_Options

from sim_instructions import *
from controller import SimState, run_simulation

X = Operator.from_label('X')
Y = Operator.from_label('Y')
Z = Operator.from_label('Z')
M0 = Operator(np.zeros((2,2)))

## 0. Set up simple qubit model

In [2]:
# operator model/solver
w = 1.
r = 0.05

operators = [-1j * w * np.pi * Z, -1j * 2 * np.pi * r * X / 2]
signals = [Constant(1.), 
           ConstantSignal(1., carrier_freq=w)]
#transformations = {'frame': -1j*2*np.pi*w*Z/2}#, 'rwa_freq_cutoff': 2*w}

# instantiate model
test_model = BaseModel(signals, operators)#, transformations=transformations)

t0 = 0.
y0 = np.array([1., 0.], dtype=complex)

test_prob = BMDE_Problem(test_model)
solver_options = DE_Options(method='scipy-RK45', atol=10**-8, rtol=10**-8)
qubit_solver = BMDE_Solver(test_prob, options=solver_options)

## 1. Evolution, snapshots, and measurement distribution evaluation

These are all fully deterministic processes

In [3]:
sim_state = SimState(t=0., de_state=y0, c_state=None, de_solver=qubit_solver)

In [4]:
inst1 = EvolveDE(10.)
inst2 = SnapShot(lambda y: y.de_state)
inst3 = EvolveDE(12.)
measurement = lambda y: np.abs(y)**2 # measurement distribution for computational basis
inst4 = EvaluateMeasurementDistribution(measurement) 

inst_list = [inst1, inst2, inst3, inst4]

In [5]:
results = run_simulation(sim_state, inst_list, shots = 10)

../controller/controller.py:54: UserWarning: More than 1 shot requested for fully deterministic simulation.
  warn('More than 1 shot requested for fully deterministic simulation.')


Warning raised above as multiple shots of a fully deterministic process were requested.

As this simulation is fully deterministic, the return type is just a single `sim_state` object.

In [6]:
type(results)

controller.SimState

Snapshot at time 10

In [7]:
results.snapshots

[(10.0, array([7.07117634e-01+0.00442011j, 9.24585750e-10-0.70708212j]))]

`de_state` and `c_state` (the probability distribution) at the end

In [8]:
print(results.de_state)
print(results.c_state)

[5.87800150e-01+0.00505716j 7.01762201e-10-0.80899037j]
[0.34553459 0.65446542]


## 2. Destructive measurements

Destructive measurements (and "measurements" in general) are non-deterministic, as they involve sampling. Destructive measurements are composite instructions: evaluate distribution -> sample distribution and delete state

In [9]:
inst1 = EvolveDE(100.)
inst2 = SnapShot(lambda y: y.de_state)
inst3 = EvolveDE(12.)
measurement = lambda y: np.abs(y)**2 # measurement distribution for computational basis
inst4 = DestructiveMeasurement(measurement)

inst_list = [inst1, inst2, inst3, inst4]

In [10]:
sim_state = SimState(t=0., de_state=y0, c_state=None, de_solver=qubit_solver)
results = run_simulation(sim_state, inst_list, shots = 10)

The returned value is now a list of sim states, as the simulation involves non-deterministic elements.

In [ ]:
print([result.c_state for result in results])
print([result.de_state for result in results])

[array([1]), array([1]), array([0]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1]), array([1])]
[None, None, None, None, None, None, None, None, None, None]


For each sim state, the `c_state` is now a sampled outcome of the measurement (this is sampled from the final distribution of the simulation in the previous section). For each simulation, the `de_state` is now `None`, as it is deleted by a `DestructiveMeasurement`.

## 3. Non-destructive measurements

Non-destructive measurements are mappings from `de_state` -> `de_state, c_state`, i.e. they update the `de_state` and also produce a classical value.

Below is a qubit measurement in the `X` basis.

In [12]:
P0 = np.array([[1, 1], [1, 1]]) / 2
P1 = np.array([[1, -1], [-1, 1]]) / 2
def x_measurement(y, seed=None):
    P0y = P0 @ y
    P1y = P1 @ y
    
    probs = np.array([ (np.abs(P0y)**2).sum(), (np.abs(P1y)**2).sum()])
    probs_norm = probs / probs.sum()
    rng = np.random.default_rng(seed)
    c_state = int(rng.choice(2, 1, p=probs_norm))
    de_state = None
    if c_state == 0:
        de_state = P0y / np.sqrt(probs[0])
    else:
        de_state = P1y / np.sqrt(probs[1])
    
    return de_state, c_state

In [13]:
inst1 = EvolveDE(10.)
inst2 = SnapShot(lambda y: y.de_state)
inst3 = NonDestructiveMeasurement(x_measurement)
inst30 = SnapShot(lambda y: y.de_state)
inst4 = EvolveDE(10.)
inst5 = NonDestructiveMeasurement(x_measurement)

inst_list = [inst1, inst2, inst3, inst30, inst4, inst5]

In [14]:
sim_state = SimState(t=0., de_state=y0, c_state=None, de_solver=qubit_solver)
results = run_simulation(sim_state, inst_list, shots = 10)#, seed=1)

State before and after measurement of first simulation run

In [ ]:
np.array([result.c_state for result in results]).sum()

5

State after measurement for each simulation run

In [ ]:
[result.snapshots[1] for result in results]

[(10.0, array([ 0.49845224+0.50154298j, -0.49845224-0.50154298j])),
 (10.0, array([ 0.49845224+0.50154298j, -0.49845224-0.50154298j])),
 (10.0, array([ 0.49845224+0.50154298j, -0.49845224-0.50154298j])),
 (10.0, array([0.50157775-0.49841725j, 0.50157775-0.49841725j])),
 (10.0, array([0.50157775-0.49841725j, 0.50157775-0.49841725j])),
 (10.0, array([0.50157775-0.49841725j, 0.50157775-0.49841725j])),
 (10.0, array([ 0.49845224+0.50154298j, -0.49845224-0.50154298j])),
 (10.0, array([ 0.49845224+0.50154298j, -0.49845224-0.50154298j])),
 (10.0, array([0.50157775-0.49841725j, 0.50157775-0.49841725j])),
 (10.0, array([0.50157775-0.49841725j, 0.50157775-0.49841725j]))]

## 4. Update signal frequencies

Demo of a model-updating sim instruction relevant to immediate feature improvements to the pulse simulator.

In this simulation:
- Initial state is in ground state of a 3 level system, and carrier freq of drive signal initially set to transition frequency between ground <-> first excited state
- First drive a pi pulse to go to first excited state
- Drive frequency is updated to transition between first and second excited state
- Another pi pulse is done to go to second excited state

Snapshots that compute the probability distribution at each step are evaluated.

In [17]:
r = 0.01
w1 = 4.
w2 = 6.

drift = 2 * np.pi * Operator([[0, 0, 0], [0, w1, 0], [0, 0, w2]])
drive = 2 * np. pi * r * Operator([[0, 1, 0], [1, 0, np.sqrt(2)], [0, np.sqrt(2), 0]])


operators = [-1j * drift, -1j * drive]
signals = [Constant(1.), 
           ConstantSignal(1., carrier_freq=w1)]
transformations = {'frame': -1j*drift}#, 'rwa_freq_cutoff': 2*w}

# instantiate model
test_model = BaseModel(signals, operators)#, transformations=transformations)

t0 = 0.
y0 = np.array([1., 0., 0.], dtype=complex)

#solver_options = {'scipy_options': {'atol': 10**-8, 'rtol': 10**-8}}
test_prob = BMDE_Problem(test_model)
solver_options = DE_Options(method='scipy-RK45', atol=10**-8, rtol=10**-8)
qutrit_solver = BMDE_Solver(test_prob, options=solver_options)

In [18]:
# evaluate probabilities of the state in the computational basis
prob_f = lambda sim_state: np.abs(sim_state.de_state)**2 / (np.abs(sim_state.de_state)**2).sum()

inst0 = SnapShot(prob_f)
inst1 = EvolveDE(0.5 / r) # pi pulse on first transition
inst2 = SnapShot(prob_f)
inst3 = UpdateSignalFrequencies([1], [w2-w1]) # update drive frequency to second transition
inst4 = EvolveDE(0.5 / r / np.sqrt(2), relative=True) # pi pulse on second transition
                                                      # relative flag means evolution time is relative to current time
inst5 = SnapShot(prob_f)

inst_list = [inst0, inst1, inst2, inst3, inst4, inst5]

In [19]:
signals = [Constant(1.), 
           ConstantSignal(1., carrier_freq=w1)]
qutrit_solver.signals = signals
sim_state = SimState(t=0., de_state=y0, c_state=None, de_solver=qutrit_solver)
results = run_simulation(sim_state, inst_list, shots = 1)

In [20]:
results.snapshots

[(0.0, array([1., 0., 0.])),
 (50.0, array([1.08511092e-06, 9.99993360e-01, 5.55530086e-06])),
 (85.35533905932738, array([1.91597946e-05, 4.62567551e-06, 9.99976215e-01]))]

## Sorting of instructions based on t value, and implicit DE evolution

The above examples are a sort of "literal" instruction implementation - i.e. instructions are specified in a list and executed in order. The above mode doesn't really "know" anything about time, and also requires all `EvolveDE` instrucitons to be explicitly stated.

In practice it will be nice to also have the following capabilities:
- `EvolveDE` instructions are never explicitly stated
- All other instructions are specified with a given time value, which can be automatically sorted and have `EvolveDE` instructions inserted.
    - This will facilitate, e.g. constructing an instruction list and then later inserting an additional snapshot.
   

In [21]:
# evaluate probabilities of the state in the computational basis
prob_f = lambda sim_state: np.abs(sim_state.de_state)**2 / (np.abs(sim_state.de_state)**2).sum()

inst0 = SnapShot(prob_f, t=0)
#inst1 = EvolveDE(0.5 / r) # pi pulse on first transition
inst2 = SnapShot(prob_f, t=0.5/r)
inst3 = UpdateSignalFrequencies([1], [w2-w1], t=0.5/r) # update drive frequency to second transition
#inst4 = EvolveDE(0.5 / r / np.sqrt(2), relative=True) # pi pulse on second transition
                                                      # relative flag means evolution time is relative to current time
inst5 = SnapShot(prob_f, t=(0.5/r + 0.5 / r / np.sqrt(2)))

inst_list = [inst0, inst2, inst3, inst5]
sorted_evolve = time_sort_and_insert_EvolveDE(inst_list)

In [22]:
sim_state = SimState(t=0., de_state=y0, c_state=None, de_solver=qutrit_solver)
results = run_simulation(sim_state, sorted_evolve, shots = 1)

In [23]:
results.snapshots

[(0.0, array([1., 0., 0.])),
 (50.0, array([1.08511092e-06, 9.99993360e-01, 5.55530086e-06])),
 (85.35533905932738, array([1.91597946e-05, 4.62567551e-06, 9.99976215e-01]))]